# MONTE CARLO NÃO-SEQUENCIAL

## IMPORTAÇÃO DE BIBLIOTECAS

In [14]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo

## LEITURA DE DADOS

In [5]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']


In [6]:
#Deixando os dados em horas e nos formatos conhecidos
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

In [8]:
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760
D_LIN['MTTF'] = 1 / D_LIN['Lambda']
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

In [9]:
Ng = len(D_GEN)
geradores = np.arange(1,Ng+1)
Nl = len(D_LIN)
linhas = np.arange(1,Nl+1)
Nc = len(D_LOAD)
cargas = np.arange(1,Nc+1)

In [12]:
#Funções Auxiliares:
def sortear_tempo_falha(mttf):
    return np.random.exponential(mttf)

def sortear_tempo_reparo(mttr):
    return np.random.exponential(mttr)

def inicializar_componentes(D_GEN, D_LIN):
    # Inicializar componentes, 1=operando, 0=falha
    estados_ger = np.ones(len(D_GEN))
    estados_lin = np.ones(len(D_LIN))

    #Sortear o tempo de falha para cada componente
    tempos_gen = [sortear_tempo_falha(mttf) for mttf in D_GEN['MTTF']]
    tempos_lin = [sortear_tempo_falha(mttf) for mttf in D_LIN['MTTF']]

    return estados_ger, estados_lin, tempos_gen, tempos_lin


In [13]:
#Parâmetros principais:
tol = 0.01 #Tolerância para  o critério de parada
NY_MAX = 100 #Máximo número de anos para simular
NY = 0 #Contador de anos

#Inicializando Índices
F_LOLE = 0
F_EENS = 0
F_LOLF = 0
F_LOLD = 0

In [ ]:
#Algoritmo de Monte Carlo Sequencial:
while NY < NY_MAX:
    NY = NY + 1
    TSIS = 0 #Inicializa o tempo acumulado do sistema

    #Inicializa os estados e tempos de residência:
    estados_gen, estados_lin, tempos_gen, tempos_lin = inicializar_componentes(D_GEN, D_LIN)
    TG = tempos_gen + tempos_lin  # Tempos globais de residência (falha ou reparo)

    while TSIS < 8760:
        # Determina o componente com o menor tempo de residência (falha ou recuperação)
        TGi = min(TG)
        if TGi > 8760:
            TGi = 8760
        
        TDUR = TGi - TSIS # Tempo de duração do estado atual
        TSIS = TGi # Atualiza o tempo acumulado do sistema

        #Atualiza o impacto do estado atual nos índies de confiabilidade
        z_ger = estados_gen
        z_lin =  estados_lin

        #Calcula o FPO
        carga_cortada = calcular_fpo(z_ger, z_lin)

        if carga_cortada > 0:
            F_LOLE += TDUR #Tempo de perda de carga (LOLE)
            F_EENS += carga_cortada * TDUR #Energia não fornecida (EENS)
            F_LOLF += 1 #Frequência de corte de carga (LOLF)
        
        



In [ ]:
####Definir os Z's de acordo com o problema!

## FPO

In [40]:
#Criando modelo Concreto:
model = ConcreteModel()

In [ ]:
#Criando Sets do Problema:
model.ger = RangeSet(Ng) #Geradores
model.line = RangeSet(Nl) #Linhas
model.carga = RangeSet(Nc) #Cargas
model.bus = RangeSet(6) #Barras

In [42]:
#Parâmtro das Cargas:
model.p_carga = Param(model.carga,initialize={cargas[i]: D_LOAD.loc[i, 'Carga'] for i in range(len(cargas))})
model.carga_bus = Param(model.carga,initialize={cargas[i]: D_LOAD.loc[i, 'Barra'] for i in range(len(cargas))})

#Parâmtros das Linhas:
model.barra_de = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Barra De'] for i in range(len(linhas))})
model.barra_para = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Barra Para'] for i in range(len(linhas))})
model.x = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Reatância'] for i in range(len(linhas))})
model.flux_max = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Capacidade'] for i in range(len(linhas))})

#Parâmetros dos Geradores:
model.ger_bus = Param(model.ger,initialize={geradores[i]: D_GEN.loc[i, 'Barra'] for i in range(len(geradores))})
model.ger_max = Param(model.ger,initialize={geradores[i]: D_GEN.loc[i, 'Cap Ind'] for i in range(len(geradores))})


In [ ]:
#Parâmetros binários que representam o estado do componente: 0:down e 1:up
model.z_ger = Param(model.ger, initialize={geradores[i]: z_ger[i] for i in range(len(geradores))})
model.z_line = Param(model.line, initialize={linhas[i]: z_line[i] for i in range(len(linhas))})

In [43]:
#Variáveis:
model.pd = Var(model.carga, domain=NonNegativeReals) #Variável de corte de carga
model.pg = Var(model.ger, domain=NonNegativeReals) #Variável de geração
model.theta = Var(model.bus, bounds=(-np.pi, np.pi), domain=Reals) #Variável de ângulo nas barras
model.flux = Var(model.line, domain=Reals) #Variável de fluxo, opcional mas facilita a implementação

In [44]:
#Restrições:
#Balanço de Potência nas Barras
def balance_power_rule(model, barra):
    PG = sum(model.pg[ger] for ger in model.ger if model.ger_bus[ger] == barra)
    PL = sum(model.p_carga[car] for car in model.carga if model.carga_bus[car] == barra)
    fluxo_sum = 0
    for line in model.line:
        if model.barra_de[line] == barra:
            fluxo_sum += model.flux[line]
        elif model.barra_para[line] == barra:
            fluxo_sum -= model.flux[line]
    PD = sum(model.pd[carga] for carga in model.carga if model.carga_bus[carga] == barra)
    return PG + PD - fluxo_sum - PL == 0
model.balance_power = Constraint(model.bus, rule=balance_power_rule)

In [ ]:
#Restrição de igualdade em que se define a geração:
def ger_rule(model, ger):
    geracao = model.ger_max[ger]*model.z_ger[ger]
    return model.pg[ger] == geracao
model.ger_rule = Constraint(model.ger, rule=ger_rule)

In [ ]:
#Restrição de igualdade em que se define os fluxos nas linhas:
def flow_rule(model, line):
    bus1 = model.barra_de[line]
    bus2 = model.barra_para[line]
    flux = (model.theta[bus1] - model.theta[bus2]) / model.x[line]
    return model.flux[line] == flux
model.flow_rule = Constraint(model.line, rule=flow_rule)

#Limites de Fluxo nas Linhas:
def flow_limit_rule(model, line):
    limite = model.flux_max[line]*model.z_line[line]
    return (-limite, model.flux[line], limite)
model.flow_limit = Constraint(model.line, rule=flow_limit_rule)

In [45]:
#Limita o corte ao valor da carga:
def load_shed_limit_rule(model, carga):
    return model.pd[carga] <= model.p_carga[carga]
model.shed_limit = Constraint(model.carga, rule=load_shed_limit_rule)

In [46]:
#Função Objetivo: Menor perda de carga possível:
def objective_function(model):
    PD_sum = sum(model.pd[carga] for carga in model.carga)
    return PD_sum
model.objective = Objective(rule=objective_function, sense=minimize)